In [ ]:
!unzip /content/448x448_top_0312.zip

Archive:  /content/448x448_top_0312.zip
   creating: 448x448_top/
  inflating: __MACOSX/._448x448_top  
   creating: 448x448_top/valid/
  inflating: __MACOSX/448x448_top/._valid  
  inflating: 448x448_top/.DS_Store   
  inflating: __MACOSX/448x448_top/._.DS_Store  
   creating: 448x448_top/train/
  inflating: __MACOSX/448x448_top/._train  
  inflating: 448x448_top/valid/.DS_Store  
  inflating: __MACOSX/448x448_top/valid/._.DS_Store  
   creating: 448x448_top/valid/Bright_Field/
  inflating: __MACOSX/448x448_top/valid/._Bright_Field  
  inflating: 448x448_top/train/_annotations.coco.json  
  inflating: __MACOSX/448x448_top/train/.__annotations.coco.json  
  inflating: 448x448_top/train/.DS_Store  
  inflating: __MACOSX/448x448_top/train/._.DS_Store  
   creating: 448x448_top/train/Bright_Field/
  inflating: __MACOSX/448x448_top/train/._Bright_Field  
   creating: 448x448_top/valid/Bright_Field/top/
  inflating: __MACOSX/448x448_top/valid/Bright_Field/._top  
  inflating: 448x448_top/va

In [ ]:
import os
import json
import cv2
import albumentations as A
import shutil

# ================= CẤU HÌNH ĐƯỜNG DẪN =================
INPUT_JSON_PATH = '/content/448x448_top/train/_annotations.coco.json'   # File json gốc
INPUT_IMAGES_DIR = '/content/448x448_top/train/Bright_Field/top'            # Folder ảnh gốc (bạn cần upload lên Colab)

OUTPUT_DIR = 'augmented_dataset'             # Folder đầu ra
OUTPUT_IMAGES_DIR = os.path.join(OUTPUT_DIR, 'images')
OUTPUT_JSON_PATH = os.path.join(OUTPUT_DIR, '_annotations_augmented.coco.json')
# ========================================================

def ensure_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)

def save_json(data, path):
    with open(path, 'w') as f:
        json.dump(data, f, indent=4)

# Định nghĩa các pipeline Augmentation
def get_transforms():
    # 1. Pipeline Lật Ngang
    t_flip = A.Compose([
        A.HorizontalFlip(p=1.0)
    ], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']))

    # 2. Pipeline Xoay 90 độ (Clockwise)
    # Mẹo: Transpose (đổi trục) + Horizontal Flip = Rotate 90 Clockwise
    # Cách này tốt hơn A.Rotate vì nó tự động đổi chiều Width/Height của ảnh mà không cắt (crop) ảnh
    t_rot90 = A.Compose([
        A.Transpose(p=1.0),
        A.HorizontalFlip(p=1.0)
    ], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']))

    return t_flip, t_rot90

def augment_data():
    ensure_dir(OUTPUT_IMAGES_DIR)
    coco_data = load_json(INPUT_JSON_PATH)

    # Khởi tạo cấu trúc JSON mới
    new_coco = {
        'info': coco_data.get('info', {}),
        'licenses': coco_data.get('licenses', []),
        'categories': coco_data['categories'],
        'images': [],
        'annotations': []
    }

    # Map annotations theo image_id
    anns_map = {}
    for ann in coco_data['annotations']:
        anns_map.setdefault(ann['image_id'], []).append(ann)

    # ID counters
    img_id_count = max([img['id'] for img in coco_data['images']]) + 1
    ann_id_count = max([ann['id'] for ann in coco_data['annotations']]) + 1 if coco_data['annotations'] else 1

    transform_flip, transform_rot90 = get_transforms()

    print(f"Bắt đầu xử lý {len(coco_data['images'])} ảnh...")

    for img_info in coco_data['images']:
        img_id = img_info['id']
        file_name = img_info['file_name']
        img_path = os.path.join(INPUT_IMAGES_DIR, file_name)

        if not os.path.exists(img_path):
            print(f"Bỏ qua (không thấy ảnh): {file_name}")
            continue

        # Đọc ảnh
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Albumentations dùng RGB

        # Lấy annotations của ảnh
        anns = anns_map.get(img_id, [])

        # Chuẩn bị dữ liệu cho Albumentations
        bboxes = []
        category_ids = []
        original_anns_lookup = [] # Để lưu lại các thông tin khác (iscrowd, area...)

        for ann in anns:
            bboxes.append(ann['bbox'])
            category_ids.append(ann['category_id'])
            original_anns_lookup.append(ann)

        # Kiểm tra xem ảnh có chứa class 2 không
        has_class_2 = 2 in category_ids

        # --- 1. LUÔN LƯU ẢNH GỐC ---
        # Save ảnh gốc sang folder mới
        out_img_name = file_name
        cv2.imwrite(os.path.join(OUTPUT_IMAGES_DIR, out_img_name), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

        new_coco['images'].append(img_info)
        new_coco['annotations'].extend(anns) # Giữ nguyên ann gốc

        # --- 2. AUGMENT NẾU CÓ CLASS 2 ---
        if has_class_2:
            aug_operations = [
                ('flip', transform_flip),
                ('rot90', transform_rot90)
            ]

            for aug_name, transform in aug_operations:
                try:
                    # Thực hiện transform
                    transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)

                    aug_img = transformed['image']
                    aug_bboxes = transformed['bboxes']
                    aug_cat_ids = transformed['category_ids']

                    # Tạo thông tin ảnh mới
                    new_filename = f"aug_{aug_name}_{file_name}"
                    new_h, new_w, _ = aug_img.shape

                    new_img_id = img_id_count
                    img_id_count += 1

                    new_img_entry = {
                        'id': new_img_id,
                        'file_name': new_filename,
                        'height': new_h,
                        'width': new_w,
                        'date_captured': img_info.get('date_captured', ''),
                        'license': img_info.get('license', 1)
                    }
                    new_coco['images'].append(new_img_entry)

                    # Lưu ảnh mới
                    cv2.imwrite(os.path.join(OUTPUT_IMAGES_DIR, new_filename), cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))

                    # Tạo annotations mới
                    for i, bbox in enumerate(aug_bboxes):
                        # Lấy lại thông tin gốc (như iscrowd, attributes) từ ann cũ
                        old_ann = original_anns_lookup[i]

                        new_ann = {
                            'id': ann_id_count,
                            'image_id': new_img_id,
                            'category_id': aug_cat_ids[i],
                            'bbox': list(bbox), # Albumentations trả về tuple, cần convert sang list
                            'area': bbox[2] * bbox[3], # Tính lại diện tích
                            'segmentation': [], # Segmentation rỗng vì khó biến đổi chính xác nếu chỉ dùng bbox transform
                            'iscrowd': old_ann.get('iscrowd', 0)
                        }
                        ann_id_count += 1
                        new_coco['annotations'].append(new_ann)

                except Exception as e:
                    print(f"Lỗi khi augment ảnh {file_name} ({aug_name}): {e}")

    # Lưu file JSON
    save_json(new_coco, OUTPUT_JSON_PATH)
    print("\nHoàn tất!")
    print(f"Dữ liệu đã lưu tại: {OUTPUT_DIR}")
    print(f"Tổng ảnh ban đầu: {len(coco_data['images'])}")
    print(f"Tổng ảnh sau khi augment: {len(new_coco['images'])}")

if __name__ == "__main__":
    augment_data()

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Bỏ qua (không thấy ảnh): S255_Image__2025-11-11__14-26-44_dark_6_crop_2_jpg.rf.24b1fbe54fa3afb4730f74cfa77897b5.jpg
Bỏ qua (không thấy ảnh): S229_Image__2025-11-05__13-21-52_dark_5_crop_5_jpg.rf.e157d68453348b64c9c65d629ecfc4d1.jpg
Bỏ qua (không thấy ảnh): S62_Image__2025-09-08__10-01-14_bright_3_crop_1_jpg.rf.158e62f80c4562b1bd57459b681cfc95.jpg
Bỏ qua (không thấy ảnh): S138_Image__2025-09-24__10-13-38_dark_1_crop_2_jpg.rf.e44f79a06a9a6ec19eb7260bbfbdfcab.jpg
Bỏ qua (không thấy ảnh): S234_Image__2025-11-05__15-05-25_bright_7_crop_8_jpg.rf.4b9a92d94be6afc26a5cb5fcd447df46.jpg
Bỏ qua (không thấy ảnh): S156_Image__2025-09-24__15-02-57_bright_2_crop_9_jpg.rf.0a0ba1bf9fadb9b5fbed93056d8f890f.jpg
Bỏ qua (không thấy ảnh): S71_Image__2025-09-22__11-01-57_bright_4_crop_1_jpg.rf.a8760b6ff998adae76bbe68c31406698.jpg
Bỏ qua (không thấy ảnh): S271_Image__2025-11-17__11-18-06_dark_4_crop_5_jpg.rf.abf18d5bb821e787c0a21fd478668282.jpg
Bỏ qua (k

In [ ]:
import os

def count_images(folder_path):
    # Danh sách các đuôi file ảnh muốn đếm (có thể thêm bớt tùy nhu cầu)
    valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp'}

    count = 0
    if not os.path.exists(folder_path):
        print(f"Lỗi: Folder '{folder_path}' không tồn tại.")
        return 0

    print(f"Đang quét folder: {folder_path} ...")

    # Duyệt qua tất cả các file trong folder
    for filename in os.listdir(folder_path):
        # Lấy đường dẫn đầy đủ
        file_path = os.path.join(folder_path, filename)

        # Chỉ đếm nếu là file (không đếm folder con)
        if os.path.isfile(file_path):
            # Lấy đuôi file và chuyển về chữ thường
            ext = os.path.splitext(filename)[1].lower()

            # Kiểm tra xem đuôi file có nằm trong danh sách ảnh không
            if ext in valid_extensions:
                count += 1

    return count

# ================= SỬ DỤNG =================
# Thay đường dẫn folder của bạn vào đây
folder_to_check = '/content/augmented_dataset/images'

num_images = count_images(folder_to_check)
print(f"✅ Tổng số lượng ảnh tìm thấy: {num_images}")

Đang quét folder: /content/augmented_dataset/images ...
✅ Tổng số lượng ảnh tìm thấy: 598


In [ ]:
import json
import os
import shutil
import random
import yaml
from pathlib import Path

# ================= CẤU HÌNH ĐẦU VÀO =================
# Đường dẫn đến dữ liệu đã Augment ở bước trước
INPUT_JSON = 'augmented_dataset/_annotations_augmented.coco.json'
INPUT_IMG_DIR = 'augmented_dataset/images'

# Đường dẫn đầu ra cho YOLO
OUTPUT_BASE = 'yolo_dataset'
# ====================================================

def convert_coco_to_yolo():
    # 1. Tạo cấu trúc thư mục
    for split in ['train', 'val']:
        os.makedirs(os.path.join(OUTPUT_BASE, 'images', split), exist_ok=True)
        os.makedirs(os.path.join(OUTPUT_BASE, 'labels', split), exist_ok=True)

    # 2. Đọc file JSON
    with open(INPUT_JSON, 'r') as f:
        data = json.load(f)

    # Tạo map category_id -> yolo_id (0, 1, 2...)
    # YOLO yêu cầu class index bắt đầu từ 0
    categories = sorted(data['categories'], key=lambda x: x['id'])
    cat_map = {c['id']: i for i, c in enumerate(categories)}
    class_names = [c['name'] for c in categories]

    print("Mapping Class ID:", cat_map)
    print("Class Names:", class_names)

    # Gom annotations theo image_id
    anns_map = {}
    for ann in data['annotations']:
        anns_map.setdefault(ann['image_id'], []).append(ann)

    # Lấy danh sách ảnh và xáo trộn để chia train/val
    images = data['images']
    random.seed(42)
    random.shuffle(images)

    # Chia 80% train, 20% val
    split_idx = int(len(images) * 0.8)
    train_imgs = images[:split_idx]
    val_imgs = images[split_idx:]

    def process_images(img_list, split_type):
        print(f"Đang xử lý {len(img_list)} ảnh cho tập {split_type}...")

        for img_info in img_list:
            img_id = img_info['id']
            file_name = img_info['file_name']

            # 1. Copy ảnh
            src_img = os.path.join(INPUT_IMG_DIR, file_name)
            dst_img = os.path.join(OUTPUT_BASE, 'images', split_type, file_name)

            if not os.path.exists(src_img):
                continue # Bỏ qua nếu không thấy file ảnh

            shutil.copy(src_img, dst_img)

            # 2. Tạo file label .txt
            img_w = img_info['width']
            img_h = img_info['height']
            anns = anns_map.get(img_id, [])

            label_file = os.path.splitext(file_name)[0] + '.txt'
            dst_label = os.path.join(OUTPUT_BASE, 'labels', split_type, label_file)

            with open(dst_label, 'w') as f_out:
                for ann in anns:
                    cat_id = ann['category_id']
                    if cat_id not in cat_map:
                        continue

                    class_idx = cat_map[cat_id]
                    bbox = ann['bbox'] # [x_min, y_min, width, height]

                    # Chuyển đổi COCO -> YOLO (Normalized xywh)
                    # x_center = (x_min + w/2) / img_w
                    # y_center = (y_min + h/2) / img_h
                    # w_norm = w / img_w
                    # h_norm = h / img_h

                    x_center = (bbox[0] + bbox[2]/2) / img_w
                    y_center = (bbox[1] + bbox[3]/2) / img_h
                    w_norm = bbox[2] / img_w
                    h_norm = bbox[3] / img_h

                    # Giới hạn giá trị trong khoảng 0-1 (đề phòng lỗi augmentation tràn viền)
                    x_center = max(0.0, min(1.0, x_center))
                    y_center = max(0.0, min(1.0, y_center))
                    w_norm = max(0.0, min(1.0, w_norm))
                    h_norm = max(0.0, min(1.0, h_norm))

                    f_out.write(f"{class_idx} {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")

    process_images(train_imgs, 'train')
    process_images(val_imgs, 'val')

    # 3. Tạo file data.yaml
    yaml_content = {
        'path': os.path.abspath(OUTPUT_BASE), # Đường dẫn tuyệt đối
        'train': 'images/train',
        'val': 'images/val',
        'nc': len(class_names),
        'names': class_names
    }

    yaml_path = os.path.join(OUTPUT_BASE, 'data.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)

    print(f"\n✅ Đã tạo xong dataset tại: {OUTPUT_BASE}")
    print(f"✅ File cấu hình YAML tại: {yaml_path}")
    return yaml_path

# Chạy chuyển đổi
yaml_file_path = convert_coco_to_yolo()

Mapping Class ID: {0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5}
Class Names: ['drill', 'Broken', 'Chipped', 'Scratched', 'Severe_Rust', 'Tip_Wear']
Đang xử lý 478 ảnh cho tập train...
Đang xử lý 120 ảnh cho tập val...

✅ Đã tạo xong dataset tại: yolo_dataset
✅ File cấu hình YAML tại: yolo_dataset/data.yaml


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.8 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

# Load model pre-trained (transfer learning)
model = YOLO('yolo11n.pt')

# Bắt đầu train
# data: đường dẫn file yaml vừa tạo ở bước 1
# epochs: số lần lặp
# imgsz: kích thước ảnh resize
results = model.train(
    data='yolo_dataset/data.yaml',
    epochs=100,
    imgsz=448,
    batch=16,
    name='my_custom_yolo_model' # Tên folder lưu kết quả
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.234 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=448, int8=False, iou=0.7, keras=False,

In [ ]:
from google.colab import drive
import os
import shutil

# Mount Google Drive vào Colab
drive.mount('/content/drive')

# Đường dẫn folder đích trên Drive nơi bạn muốn lưu file
# (Bạn có thể đổi tên 'Backup_YOLO_Data' thành tên tùy thích)
dest_folder = '/content/drive/MyDrive/Backup_YOLO_Data'

# Tạo folder trên Drive nếu chưa có
if not os.path.exists(dest_folder):
    os.makedirs(dest_folder)
    print(f"Đã tạo folder mới trên Drive: {dest_folder}")
else:
    print(f"Lưu vào folder có sẵn trên Drive: {dest_folder}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Đã tạo folder mới trên Drive: /content/drive/MyDrive/Backup_YOLO_Data


In [ ]:
# Danh sách các folder bạn muốn lưu
# 'augmented_dataset': Dữ liệu ảnh + json đã augment
# 'yolo_dataset': Dữ liệu đã chia train/val cho YOLO
# 'runs': Kết quả training (model weights, biểu đồ...)
folders_to_save = ['augmented_dataset', 'yolo_dataset', 'runs']

for folder in folders_to_save:
    # Kiểm tra xem folder có tồn tại không
    if os.path.exists(folder):
        print(f"Đang xử lý folder: {folder}...")

        # 1. Nén folder thành file .zip
        zip_filename = f"{folder}.zip"
        print(f"  -> Đang nén thành {zip_filename} (có thể mất vài giây)...")
        os.system(f"zip -r -q {zip_filename} {folder}")

        # 2. Copy file zip sang Google Drive
        dest_path = os.path.join(dest_folder, zip_filename)
        print(f"  -> Đang copy sang Drive: {dest_path}")
        shutil.copy(zip_filename, dest_path)

        print("  -> ✅ Xong!")
    else:
        print(f"⚠️ Không tìm thấy folder '{folder}', bỏ qua.")

print(f"\n🎉 Đã lưu toàn bộ dữ liệu vào: {dest_folder}")

Đang xử lý folder: augmented_dataset...
  -> Đang nén thành augmented_dataset.zip (có thể mất vài giây)...
  -> Đang copy sang Drive: /content/drive/MyDrive/Backup_YOLO_Data/augmented_dataset.zip
  -> ✅ Xong!
Đang xử lý folder: yolo_dataset...
  -> Đang nén thành yolo_dataset.zip (có thể mất vài giây)...
  -> Đang copy sang Drive: /content/drive/MyDrive/Backup_YOLO_Data/yolo_dataset.zip
  -> ✅ Xong!
Đang xử lý folder: runs...
  -> Đang nén thành runs.zip (có thể mất vài giây)...
  -> Đang copy sang Drive: /content/drive/MyDrive/Backup_YOLO_Data/runs.zip
  -> ✅ Xong!

🎉 Đã lưu toàn bộ dữ liệu vào: /content/drive/MyDrive/Backup_YOLO_Data
